Author: Irsyad Adam

GOAL: extract the summary statistics of every gwas study relating to "heart", "cardiomyopathy", "cardio"... etc

GWAS Catalog API Documentation: https://www.ebi.ac.uk/gwas/rest/docs/api

GWAS API Endpoints Documentation: https://www.ebi.ac.uk/gwas/rest/api

API Endpoint: http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/ + 'Valid GWAS ID'

In [1]:
import pandas as pd
import os
import requests
from tqdm import tqdm
import ast

In [6]:
#Final Product: 

# Step 3: run MAGMA using the following command, this takes a GWAS file ${trait}.pval,
# which at least has the columns: SNP, P, N, which corresponds to the SNP id
# (matched to the ${magma_dir}/g1000_eur.bim), p-value, sample size. For example,
# <trait>.pval file looks like
#
# CHR     BP      SNP             P           N
# 1       717587  rs144155419     0.453345    279949
# 1       740284  rs61770167      0.921906    282079
# 1       769223  rs60320384      0.059349    281744

In [23]:
os.getcwd()

'/Users/pinglab/Documents/irsyadadam/double_injury_scdrs/gwas-catalog-wrapper'

In [42]:
args = "heart_failure"

folder_name = args
query_file = "%s/%s" % (args, os.listdir(folder_name)[0])
index = pd.read_csv(query_file)
index


,First Author,PubMed ID,Study accession,Publication date,Journal,Title,Trait(s),Reported trait,Data access
0,Astle WJ,27863252,GCST004623,2016-11-17,Cell,The Allelic Landscape of Human Blood Cell Trai...,neutrophil percentage of granulocytes,Neutrophil percentage of granulocytes,FTP Download or API access
1,Astle WJ,27863252,GCST004624,2016-11-17,Cell,The Allelic Landscape of Human Blood Cell Trai...,sum of eosinophil and basophil counts,Sum eosinophil basophil counts,FTP Download or API access
2,Astle WJ,27863252,GCST004625,2016-11-17,Cell,The Allelic Landscape of Human Blood Cell Trai...,monocyte count,Monocyte count,FTP Download or API access
3,Astle WJ,27863252,GCST004600,2016-11-17,Cell,The Allelic Landscape of Human Blood Cell Trai...,eosinophil percentage of leukocytes,Eosinophil percentage of white cells,FTP Download or API access
4,Astle WJ,27863252,GCST004602,2016-11-17,Cell,The Allelic Landscape of Human Blood Cell Trai...,mean corpuscular volume,Mean corpuscular volume,FTP Download or API access
...,...,...,...,...,...,...,...,...,...
30403,de Lange KM,28067908,GCST004133,2017-01-09,Nat Genet,Genome-wide association study implicates immun...,ulcerative colitis,Ulcerative colitis,FTP Download or API access
30404,de Lange KM,28067908,GCST004131,2017-01-09,Nat Genet,Genome-wide association study implicates immun...,inflammatory bowel disease,Inflammatory bowel disease,FTP Download or API access
30405,Morris AP,22885922,GCST005047,2012-09-01,Nat Genet,Large-scale association analysis provides insi...,type 2 diabetes mellitus,Type 2 diabetes,FTP Download or API access
30406,Michailidou K,29059683,GCST004988,2017-10-23,Nature,Association analysis identifies 65 new breast ...,breast carcinoma,Breast cancer,FTP Download or API access


In [166]:
#MAIN FUNCTION
def extract_gwas(GWAS_ID) -> None:
    """
    grabs .tsv summary statistics associated with gwas id

    ARGS:
        GWAS_ID is the gwas id of the research

    RETURNS: 
        None
    """
    #get the url
    endpoint = find_endpoint(GWAS_ID)
    print(GWAS_ID, endpoint)
    summary_stats_link = ""

    #if endpointis a range of endpoints
    if len(endpoint) > len(GWAS_ID):
        url = 'http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/' + endpoint + "/" + GWAS_ID + "/"


    #else if single gwas id endpoint
    elif endpoint == GWAS_ID:
        url = 'http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/' + endpoint + "/"
    
    print(url)
    response = requests.get(url=url)
    tsv_file = extract_link(response)
    if tsv_file == None:
            url = url + "harmonised/"
            response = requests.get(url=url)
            tsv_file = extract_link(response)
    summary_stats_link = url + tsv_file


    #TODO: WGET IS SLOOWWWWWW (~2 hours to download)
    cmd = "wget %s" % summary_stats_link
    print(cmd)
    os.system(cmd)

In [167]:
def extract_link(response: requests.Response) -> str:
    """
    extracts the link from a response from def extract_gwas

    ARGS: 
        response is the http response object

    RETURNS:
        string associated with the link in the response object
    """
    txts = response.text

    #iterate through the lines for the file
    #find first occurance of .tsv file
    #get the link
    for i in txts.splitlines():
        if ".tsv" in i:
            start = '>'
            end = '<'
            link = (i.split(start))[1].split(end)[0]
            return link
    
            

In [168]:
def find_endpoint(GWAS_ID):
    df = pd.read_csv("gwas_catalog_index/index.csv", header = 0)

    #get the index
    ID = int(GWAS_ID[4:])

    endpoint = ""

    #iterate through the id range column
    for i in range(len(df["id_range"])):

        range_ids = ast.literal_eval(df["id_range"][i])
        # CASE1: if the id is in the range column, and range column only has 1 variable
        if len(range_ids) == 1 and range_ids[0] == ID:  
            endpoint = df["endpoints"][i]
            break

        #CASE2 : if id is in between the range, and range column has 2 values
        elif len(range_ids) > 1 and range_ids[0] <= ID <= range_ids[1]:
            endpoint = df["endpoints"][i]
            break
    return str(endpoint)

In [169]:
extract_gwas("GCST004623")

GCST004623 GCST004001-GCST005000
http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST004001-GCST005000/GCST004623/
wget http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST004001-GCST005000/GCST004623/harmonised/27863252-GCST004623-EFO_0007994-Build37.f.tsv.gz


--2022-07-05 15:29:53--  http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST004001-GCST005000/GCST004623/harmonised/27863252-GCST004623-EFO_0007994-Build37.f.tsv.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1105907181 (1.0G) [application/octet-stream]
Saving to: ‘27863252-GCST004623-EFO_0007994-Build37.f.tsv.gz’

     0K .......... .......... .......... .......... ..........  0%  176K 1h42m
    50K .......... .......... .......... .......... ..........  0%  170K 1h43m
   100K .......... .......... .......... .......... ..........  0%  338K 87m1s
   150K .......... .......... .......... .......... ..........  0%  336K 78m40s
   200K .......... .......... .......... .......... ..........  0%  336K 73m38s
   250K .......... .......... .......... .......... ..........  0%  356K 69m48s
   300K .......... .......... .......... .......